In [47]:
import numpy as np
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia


In [172]:
d = pd.read_csv("data/planets.csv")

# Print the first few rows of the DataFrame
print(d.info())

Gaia.ROW_LIMIT = 10000

# print the row with the planet name: "Earth"

planet = d[d["pl_name"] == "WASP-177 b"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5514 entries, 0 to 5513
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pl_name         5514 non-null   object 
 1   hostname        5514 non-null   object 
 2   sy_snum         5514 non-null   int64  
 3   sy_pnum         5514 non-null   int64  
 4   disc_facility   5514 non-null   object 
 5   disc_telescope  5514 non-null   object 
 6   pl_orbper       5271 non-null   float64
 7   pl_orbsmax      5217 non-null   float64
 8   pl_rade         5495 non-null   float64
 9   pl_radj         5494 non-null   float64
 10  pl_bmasse       5488 non-null   float64
 11  pl_bmassj       5488 non-null   float64
 12  pl_bmassprov    5514 non-null   object 
 13  pl_dens         5402 non-null   float64
 14  pl_orbeccen     4749 non-null   float64
 15  pl_insol        3834 non-null   float64
 16  pl_eqt          4072 non-null   float64
 17  pl_orbincl      4224 non-null   f

In [177]:
target_ra = planet["ra"].values[0]
target_dec = planet["dec"].values[0]
target_distance = planet["sy_dist"].values[0]

target_coord = SkyCoord(ra=target_ra * u.deg, dec=target_dec * u.deg, distance=target_distance * u.pc, frame='icrs')

target_ra = target_coord.ra.deg  # Right Ascension in degrees
target_dec = target_coord.dec.deg  # Declination in degrees
target_distance = target_coord.distance.pc # Distance in parsecs

print(target_coord.cartesian.xyz.value)

print(target_coord)

[159.90376053 -75.25526407  -5.66018739]
<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (334.7969885, -1.8344301, 176.818)>


In [190]:
from astropy.coordinates import Distance

search_distance = 1000  # distance in parsecs around the target planet
search_distance = min(search_distance, target_distance)
search_radius = np.arcsin(
    search_distance / np.sqrt((target_distance - search_distance) ** 2 + search_distance ** 2)) * 180 / np.pi
max_mag = 12

query = f"""
SELECT TOP 30000
  source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag, teff_gspphot
FROM gaiadr3.gaia_source
WHERE
  parallax IS NOT NULL
  AND ABS(1000/parallax - {target_distance}) < {search_distance}
  AND 
  1=CONTAINS(
    POINT('ICRS',ra,dec),
    CIRCLE('ICRS',{target_ra},{target_dec},{search_radius})
  )
  AND phot_g_mean_mag < {max_mag}
  ORDER BY distance_gspphot DESC 
"""

# print(query)
job = Gaia.launch_job_async(query)
raw_data = job.get_results()

star_data = []

# count the number of stars
print(len(raw_data))

raw_data.info()
data_df = raw_data.to_pandas()  # convert the data to a pandas dataframe
data_df["dist_earth"] = data_df.apply(lambda row: 1000 / row["parallax"],
                                      axis=1)  # calculate the distance from the parallax

target_coord = SkyCoord(ra=target_ra * u.deg, dec=target_dec * u.deg, distance=target_distance * u.pc, frame='icrs')
target_x, target_y, target_z = target_coord.cartesian.xyz.value

# calculate the cartesian coordinates with the target planet as the origin
star_spheric = SkyCoord(
    ra=data_df["ra"].values * u.deg,
    dec=data_df["dec"].values * u.deg,
    distance=data_df["dist_earth"].values * u.pc,
    frame='icrs'
)
data_df["x"] = data_df["dist_earth"] * np.cos(data_df["dec"]) * np.cos(data_df["ra"] - target_ra)
data_df["y"] = data_df["dist_earth"] * np.cos(data_df["dec"]) * np.sin(data_df["ra"] - target_ra)
data_df["z"] = data_df["dist_earth"] * np.sin(data_df["dec"])

# calculate the spherical coordinates with the target planet as the origin
star_cartesian = SkyCoord(
    x=data_df["x"].values - target_x,
    y=data_df["y"].values - target_y,
    z=data_df["z"].values - target_z,
    unit='pc',
    representation_type='cartesian'
)
data_df["ra_star"] = star_cartesian.spherical.lon.value
data_df["dec_star"] = star_cartesian.spherical.lat.value
data_df["dist_star"] = star_cartesian.spherical.distance.value

distance = Distance(parallax=data_df['parallax'].values * u.mas)

# Add the absolute magnitude to the dataframe
data_df['absolute_magnitude'] = data_df['phot_g_mean_mag'] - 5 * np.log10(distance.pc) + 5

data_df['relative_magnitude'] = data_df['absolute_magnitude'] + 5 * np.log10(data_df['dist_star']) - 5

data_df = data_df.query(f"dist_star < {search_distance}")  # filter out stars that are too far away
display(data_df)


INFO: Query finished. [astroquery.utils.tap.core]
30000
<Table length=30000>
      name       dtype    unit                                   description                                 n_bad
--------------- ------- -------- --------------------------------------------------------------------------- -----
      SOURCE_ID   int64                   Unique source identifier (unique within a particular Data Release)     0
             ra float64      deg                                                             Right ascension     0
            dec float64      deg                                                                 Declination     0
       parallax float64      mas                                                                    Parallax     0
           pmra float64 mas / yr                                  Proper motion in right ascension direction     0
          pmdec float64 mas / yr                                      Proper motion in declination direction     0
pho

,SOURCE_ID,ra,dec,parallax,pmra,pmdec,phot_g_mean_mag,teff_gspphot,dist_earth,x,y,z,ra_star,dec_star,dist_star,absolute_magnitude,relative_magnitude
0,2219399031381814656,330.878952,66.683193,18.746683,66.292745,19.697465,10.721334,NaN,53.342770,28.865937,-28.353568,-34.760380,160.306448,-11.809573,142.188261,7.085957,12.850275
10,2804137761912113536,15.195209,23.948449,24.765161,129.928196,8.482380,10.719999,NaN,40.379305,10.151904,11.341100,-37.400622,149.960677,-10.397230,175.874967,7.689205,13.915225
32,5971230054563227904,253.030932,-38.363685,7.062022,15.468499,7.347370,10.716130,NaN,141.602495,111.067590,-9.424149,-87.329506,126.569511,-44.895518,115.709053,4.960776,10.277612
37,6561402850697688064,331.117431,-46.687664,3.577812,13.537370,-18.126724,10.716166,NaN,279.500400,217.544573,-129.801508,-118.094881,316.580046,-54.784878,137.620189,3.484253,9.177664
42,6680530670663621248,306.580697,-41.629069,13.856582,-66.113615,49.704605,10.716505,NaN,72.167868,50.793099,2.951444,51.181551,144.368369,22.948838,145.782060,6.424786,12.243306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29964,4552121973158182144,265.879974,18.994919,4.260135,8.654231,13.323202,9.778230,NaN,234.734342,227.719703,45.692780,34.001736,60.720516,15.962173,144.223783,2.925347,8.720531
29974,6664464851576141824,290.659598,-44.459039,8.378172,2.890212,-18.125340,3.945255,NaN,119.357778,104.775873,-16.381595,-54.771699,133.118093,-31.337659,94.430575,-1.438998,3.436565
29992,389830958573984768,11.011148,46.964667,4.728666,-15.487716,-4.972830,8.623328,NaN,211.476145,204.564107,-41.850183,33.529811,36.795783,35.095338,68.163788,1.997021,6.164790
29993,1421801339399506176,266.106420,57.108027,7.145471,7.107872,-71.323067,8.572968,NaN,139.948795,108.093300,48.852171,74.262839,112.658809,30.722041,156.443822,2.843123,8.814915


In [203]:
import math
import plotly.graph_objs as go

print(target_x, target_y, target_z)

max_range = max(
    np.max(data_df["x"].values) - target_x, np.max(data_df["y"].values) - target_y,
    np.max(data_df["z"].values) - target_z,
    target_x - np.min(data_df["x"].values), target_y - np.min(data_df["y"].values),
    target_z - np.min(data_df["z"].values)
)

# Set limits for each axis centered on the target
x_limits = [target_x - max_range, target_x + max_range]
y_limits = [target_y - max_range, target_y + max_range]
z_limits = [target_z - max_range, target_z + max_range]

# Assuming `data_df` contains your star data along with relative magnitudes
# Let's say you have a 'relative_magnitude' column in the `data_df` DataFrame

# Normalize the relative magnitudes for color mapping
min_magnitude = data_df['relative_magnitude'].min()
max_magnitude = data_df['relative_magnitude'].max()

# Scale the relative magnitudes to a range [0, 1] for color interpolation
normalized_magnitude = (data_df['relative_magnitude'] - min_magnitude) / (max_magnitude - min_magnitude)

# Create a custom color array transitioning from white to blue
# Define white (1, 1, 1) and blue (0, 0, 1) in RGB format
# colors = [
#     f'rgba({int(255 * (1 - norm))},{int(255 * (1 - norm))},{255}, {255 * (1 - norm)})' for norm in normalized_magnitude
# ]

def kelvin_to_rgb(temp_kelvin):
    if math.isnan(temp_kelvin):
        return 255, 255, 255
    
    # Ensure the temperature is within the typical range for visible spectrum
    temp_kelvin = max(1000, min(temp_kelvin, 40000)) / 100

    # Calculate red
    if temp_kelvin <= 66:
        red = 255
    else:
        red = 329.698727446 * ((temp_kelvin - 60) ** -0.1332047592)
        red = max(0, min(255, red))

    # Calculate green
    if temp_kelvin <= 66:
        green = 99.4708025861 * np.log(temp_kelvin) - 161.1195681661
        green = max(0, min(255, green))
    else:
        green = 288.1221695283 * ((temp_kelvin - 60) ** -0.0755148492)
        green = max(0, min(255, green))

    # Calculate blue
    if temp_kelvin >= 66:
        blue = 255
    else:
        if temp_kelvin <= 19:
            blue = 0
        else:
            blue = 138.5177312231 * np.log(temp_kelvin - 10) - 305.0447927307
            blue = max(0, min(255, blue))

    return int(red), int(green), int(blue)

# Create the central target trace
target_trace = go.Scatter3d(
    x=[target_x], y=[target_y], z=[target_z],
    mode='markers',
    marker=dict(size=10, color='red'),
    name='Central Target'
)

# Create the Earth trace
earth_trace = go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode='markers',
    marker=dict(size=10, color='green'),
    name='Earth'
)

colors = [f'rgba{kelvin_to_rgb(teff)}' for teff in data_df['teff_gspphot']]

# Create the star cloud trace with the color array
star_trace = go.Scatter3d(
    x=data_df["x"].values, y=data_df["y"].values, z=data_df["z"].values,
    mode='markers',
    marker=dict(size=2, color=colors),  # Use custom colors for stars
    name='Stars'
)

print('count of stars:', len(data_df))

# Define layout with manually set axis limits for equal scaling
layout = go.Layout(
    title='3D Star Field Around Target (Equal Scaling)',
    scene=dict(
        xaxis=dict(title='X (parsecs)', range=x_limits),
        yaxis=dict(title='Y (parsecs)', range=y_limits),
        zaxis=dict(title='Z (parsecs)', range=z_limits),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)  # Enforces equal scaling on all axes
    ),
)

# Create the figure
fig = go.Figure(data=[target_trace, star_trace, earth_trace], layout=layout)

# Show the plot
fig.show()

159.90376053156908 -75.25526407246281 -5.660187386166147
count of stars: 3923
